# Chapter 06: Multi-Agents

In our discussions so far, we've dived into the world of AI agents and AI agent workflows, exploring how we can leverage LLMs to utilize various tools effectively. This foundational knowledge has set the stage for a natural progression: enabling LLMs to collaborate with other LLMs. By establishing a structured mechanism for these interactions, we can significantly enhance the capabilities and applications of our AI systems.  

To facilitate this collaboration, we can utilize a familiar structure, which we have already employed for tool usage:
```
Thought: <thought process on how to respond to the prompt>

Tool: <name of the tool to use>

Tool Input: <input of the tool to use>
```

By turning other LLMs into tools, we ensure that our workflow remains streamlined and efficient. This approach simplifies the decision-making process for the primary LLM, as it doesn't need to distinguish between calling a tool and calling another LLM - it simply calls a tool. The neat encapsulation of LLMs as tools allows us to maintain clarity and uniformity in the responses we seek from these interactions.  For our showcase, we will demonstrate this concept by transforming an existing workflow into a tool. As we have previously defined the inputs that a workflow expects, this transition will be smooth and illustrative of the powerful potential of multi-agent environments.

In [1]:
import sys
from assets.tools.earthquake import count_earthquakes, query_earthquakes, USGeopoliticalSurveyEarthquakeAPI
from typing import Any
from pydantic import BaseModel, Field
from loguru import logger
from language_models.agent import (
    Agent,
    OutputType,
    PromptingStrategy,
    WorkflowLLMStep,
    WorkflowFunctionStep,
    WorkflowTransformationStep,
    WorkflowStateManager,
)
from language_models.tools import Tool, current_date
from language_models.models.llm import OpenAILanguageModel
from language_models.proxy_client import ProxyClient
from language_models.settings import settings

In [2]:
logger.remove()
logger.add(sys.stderr, format="{message}", level="INFO")

proxy_client = ProxyClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
)

In [3]:
llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model="gpt-4",
    max_tokens=500,
    temperature=0.2,
)

To integrate our workflow as a tool for use by an LLM, we simply add a new function that converts the workflow into an LLM-compatible tool, maintaining the defined inputs and utilizing the existing workflow execution structure.

In [4]:
class WorkflowOutput(BaseModel):
    """Class that represents the workflow output."""

    inputs: dict[str, Any]
    output: (
        str
        | int
        | float
        | dict[str, Any]
        | BaseModel
        | list[str]
        | list[int]
        | list[float]
        | list[dict[str, Any]]
        | list[BaseModel]
        | None
    )

class Workflow(BaseModel):
    """Class that implements a workflow.

    Attributes:
        name: The name of the workflow.
        description: The description of what the workflow does.
        steps: The steps of the workflow.
        inputs: The workflow inputs.
        output: The name of the step value to output.
    """

    name: str
    description: str
    steps: list[WorkflowLLMStep | WorkflowFunctionStep | WorkflowTransformationStep]
    inputs: type[BaseModel]
    output: str
    verbose: bool

    def invoke(self, inputs: dict[str, Any]) -> WorkflowOutput:
        """Runs the workflow."""
        _ = self.inputs.model_validate(inputs)
        state_manager = WorkflowStateManager(state=inputs)
        for step in self.steps:
            output = step.invoke(state_manager.state, self.verbose)
            state_manager.update(step.name, output)

        output = state_manager.state.get(self.output)
        if self.verbose:
            logger.opt(colors=True).success(f"<b><fg #32A467>Workflow Output</fg #32A467></b>: {output}")

        return WorkflowOutput(inputs=inputs, output=output)

    def as_tool(self) -> Tool:
        """Converts the workflow into an LLM tool."""
        return Tool(
            function=lambda **inputs: self.invoke(inputs).output,
            name=self.name,
            description=self.description,
            args_schema=self.inputs,
        )

In this setup, we first define individual workflows and tools that handle specific tasks, such as extracting numbers from text and querying earthquake data. By converting these workflows into tools, we can easily integrate them into an AI agent. Finally, we combine these agents into a chat agent that can leverage the capabilities of all the defined workflows and tools.

In [5]:
system_prompt = """You are an AI assistant designed to help users with a variety of tasks.

Extract all numbers from the user's input text."""

extractor_agent = Agent.create(
    llm=llm,
    system_prompt=system_prompt,
    prompt="{prompt}",
    prompt_variables=["prompt"],
    output_type=OutputType.ARRAY_INTEGER,
    prompting_strategy=PromptingStrategy.SINGLE_COMPLETION,
    verbose=True,
)

agent_step = WorkflowLLMStep(name="numbers", agent=extractor_agent)

class Function(BaseModel):
    numbers: list[int]

function_step = WorkflowFunctionStep(name="sort", inputs=Function, function=lambda numbers: sorted(numbers))

filter_step = WorkflowTransformationStep(name="numbers_greater_10", input_field="sort", transformation="filter", function=lambda number: number > 10)

In [6]:
class Prompt(BaseModel):
    prompt: str = Field(description="The user prompt")

extract_numbers_workflow = Workflow(
    name="Find numbers greater than 10",
    description="Extracts numbers from a given text",
    steps=[agent_step, function_step, filter_step],
    inputs=Prompt,
    output="numbers_greater_10",
    verbose=True,
)

In [7]:
query_earthquakes_tool = Tool(
    function=query_earthquakes,
    name="Query Earthquakes",
    description="Use this tool to search recent earthquakes",
    args_schema=USGeopoliticalSurveyEarthquakeAPI,
)

count_earthquakes_tool = Tool(
    function=count_earthquakes,
    name="Count Earthquakes",
    description="Use this tool to count and aggregate recent earthquakes",
    args_schema=USGeopoliticalSurveyEarthquakeAPI,
)

In [8]:
system_prompt = "You are an United States Geological Survey expert who can answer questions regarding earthquakes."

earthquake_agent = Agent.create(
    llm=llm,
    system_prompt=system_prompt,
    prompt="{question}",
    prompt_variables=["question"],
    output_type=OutputType.STRING,
    tools=[current_date, count_earthquakes_tool, query_earthquakes_tool],
    prompting_strategy=PromptingStrategy.CHAIN_OF_THOUGHT,
    verbose=True,
)

In [9]:
class EarthquakeQuery(BaseModel):
    question: str = Field(description="The earthquake related user question")

earthquake_workflow = Workflow(
    name="Earthquake Agent",
    description="Allows you to answer earthquake related questions",
    inputs=EarthquakeQuery,
    output="earthquake",
    steps=[WorkflowLLMStep(name="earthquake", agent=earthquake_agent)],
    verbose=True,
)

In [10]:
agent = Agent.create(
    llm=llm,
    system_prompt=system_prompt,
    prompt="{prompt}",
    prompt_variables=["prompt"],
    tools=[extract_numbers_workflow.as_tool(), earthquake_workflow.as_tool()],
    output_type=OutputType.STRING,
    prompting_strategy=PromptingStrategy.CHAIN_OF_THOUGHT,
    verbose=True,
)

By invoking the agent with the prompt about today's earthquakes, we instruct it to use the earthquake agent encapsulated as a tool.

In [11]:
output = agent.invoke({"prompt": "How many earthquakes happened today?"})

Thought: The user is asking about the number of earthquakes that occurred today. I can use the Earthquake Agent tool to get this information.
Tool: Earthquake Agent
Tool Input: {'question': 'How many earthquakes happened today?'}
Use LLM: earthquake
Inputs: {'question': 'How many earthquakes happened today?'}
Prompt: How many earthquakes happened today?
Thought: To answer this question, I need to count the number of earthquakes that occurred today. I can use the "Count Earthquakes" tool for this. I need to set the start_time to the beginning of today and the end_time to the current time.
Tool: Current Date
Tool Input: {}
Tool Output: 2024-08-15 13:25:12.776357
Thought: Now that I have the current date and time, I can use it as the end_time in the "Count Earthquakes" tool. The start_time will be the beginning of today, which is at 00:00:00 hours of the current date.
Tool: Count Earthquakes
Tool Input: {'start_time': '2024-08-15T00:00:00', 'end_time': '2024-08-15T13:25:12'}
Tool Output: 

In [12]:
print(output.final_answer)

There were 113 earthquakes today.


By invoking the agent with the prompt about a hiking trip, we instruct it to use the numbers extraction workflow encapsulated as a tool.

In [13]:
prompt = """Extract all numbers > 10 from this context:

Last weekend, six of us went on a 15-kilometer hike, starting at 7 AM.

By noon, we had covered 10 kilometers and reached Mount Elbert's 4,401-meter summit by 2 PM, with a temperature of 12°C.

We camped 5 kilometers away by 6 PM with 12 others and returned home by 5 PM the next day."""

output = agent.invoke({"prompt": prompt})

Thought: The user wants to extract all numbers greater than 10 from the given text. I can use the "Find numbers greater than 10" tool to get this information.
Tool: Find numbers greater than 10
Tool Input: {'prompt': "Last weekend, six of us went on a 15-kilometer hike, starting at 7 AM. By noon, we had covered 10 kilometers and reached Mount Elbert's 4,401-meter summit by 2 PM, with a temperature of 12°C. We camped 5 kilometers away by 6 PM with 12 others and returned home by 5 PM the next day."}
Use LLM: numbers
Inputs: {'prompt': "Last weekend, six of us went on a 15-kilometer hike, starting at 7 AM. By noon, we had covered 10 kilometers and reached Mount Elbert's 4,401-meter summit by 2 PM, with a temperature of 12°C. We camped 5 kilometers away by 6 PM with 12 others and returned home by 5 PM the next day."}
Prompt: Last weekend, six of us went on a 15-kilometer hike, starting at 7 AM. By noon, we had covered 10 kilometers and reached Mount Elbert's 4,401-meter summit by 2 PM, wit

In [14]:
print(output.final_answer)

The numbers greater than 10 in the given context are 12, 12, 15, and 4401.
